In [14]:
# Mount google drive here ..
import os
from google.colab import drive
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')


input_path = "/content/drive/MyDrive/ML/BERT/train.csv"
output_path = "/content/drive/MyDrive/ML/train_preprocess.csv"

In [15]:
import pandas as pd
pd.set_option('display.max_colwidth', 2)

In [16]:
data = pd.read_csv(input_path)
data.head()

,id,text,class
0,dee6dfc5,"Đến quán 2 lần thôi , rất là thích !\nQuán tuy nằm trong hẻm nhưng rất dễ tìm , quán rộng_rãi thoáng mát sạch_sẽ .\nNhân_viên phục_vụ nhiệt_tình , siêu dễ_thương .\nThực_đơn phong_phú đa_dạng , giá siêu rẻ .\nThức_ăn nêm nếm hợp_khẩu vị , ngon_miệng .\nDesert ngọt vừa , không quá ngấy , rất ngon .\nTrà sữa , uyên_ương thơm và đúng vị mình đã từng uống ở HongKong .\nNói_chung điểm 10 cho chất_lượng , thích quán lắm !",1
1,800813f5,Đến quán vào tối chủ_nhật . Có band hát . Khá ổn . Tuy_nhiên không_gian tầng 1 hơi hẹp . Ngồi cả ra vỉa_hè . Đồ uống ko ngon lămz,0
2,6553e47f,Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi gần 15p mới mang được ly cà_phê ra . Ông này chỉ khéo quảng_bá chứ với mình chắc không quay lại_nữa .,0
3,b45a1ff1,"Ko gian bé_tí , quán chật_chội , đông người nên phục_vụ chưa tốt . Đồ uống bình_thường ( đặc_biệt khi so với giá_cả ) và ko có gì đặc_sắc cả .",0
4,f92694b0,"Khi mình order , đặt bánh thì nhận được sự tiếp_đãi , tư_vấn và nói_chuyện rất gần_gũi , rất nhiệt_tình .\nBánh thì rất tuyệt , nhân béo , rau_câu không dùng phẩm . Sau_này có dịp sinh_nhật , mình sẽ order mẫu lớn .\nChúc các bạn làm_ăn phát_đạt !",1


In [17]:
# Preprocess text
from bs4 import BeautifulSoup
import string, re


def preprocess_text(text):
    # Remove HTML
    soup = BeautifulSoup(text, "html5lib")
    text = soup.get_text(separator=" ")

    # Remove icon
    icon_re = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    text = icon_re.sub(r'', text)

    # Remove punctuation
    punctuation = string.punctuation.replace("_", "")  # Keep _ symbol
    text = "".join([i if i not in punctuation else " " for i in text])

    # Remove duplicate characters
    text = re.sub(r'(.)\1{1,}', r'\1', text)

    # Fix broken unicode format (big-brain part)
    text = re.sub(b"( )(\xcc[\x80-\xa9])( )", br'\2', text.encode('utf-8')).decode('utf-8')

    # Remove new_line and strip
    text = text.replace("\n", " ").strip()

    # Remove double space
    text = re.sub(" +", " ", text)

    # Lower text
    text = text.lower()

    # Return
    return text


In [18]:
data['preprocess_text'] = data['text'].apply(lambda x: preprocess_text(x))
data.head(10)

,id,text,class,preprocess_text
0,dee6dfc5,"Đến quán 2 lần thôi , rất là thích !\nQuán tuy nằm trong hẻm nhưng rất dễ tìm , quán rộng_rãi thoáng mát sạch_sẽ .\nNhân_viên phục_vụ nhiệt_tình , siêu dễ_thương .\nThực_đơn phong_phú đa_dạng , giá siêu rẻ .\nThức_ăn nêm nếm hợp_khẩu vị , ngon_miệng .\nDesert ngọt vừa , không quá ngấy , rất ngon .\nTrà sữa , uyên_ương thơm và đúng vị mình đã từng uống ở HongKong .\nNói_chung điểm 10 cho chất_lượng , thích quán lắm !",1,đến quán 2 lần thôi rất là thích quán tuy nằm trong hẻm nhưng rất dễ tìm quán rộng_rãi thoáng mát sạch_sẽ nhân_viên phục_vụ nhiệt_tình siêu dễ_thương thực_đơn phong_phú đa_dạng giá siêu rẻ thức_ăn nêm nếm hợp_khẩu vị ngon_miệng desert ngọt vừa không quá ngấy rất ngon trà sữa uyên_ương thơm và đúng vị mình đã từng uống ở hongkong nói_chung điểm 10 cho chất_lượng thích quán lắm
1,800813f5,Đến quán vào tối chủ_nhật . Có band hát . Khá ổn . Tuy_nhiên không_gian tầng 1 hơi hẹp . Ngồi cả ra vỉa_hè . Đồ uống ko ngon lămz,0,đến quán vào tối chủ_nhật có band hát khá ổn tuy_nhiên không_gian tầng 1 hơi hẹp ngồi cả ra vỉa_hè đồ uống ko ngon lămz
2,6553e47f,Phục_vụ lâu quá mặc_dù khách rất vắng .\nĐợi gần 15p mới mang được ly cà_phê ra . Ông này chỉ khéo quảng_bá chứ với mình chắc không quay lại_nữa .,0,phục_vụ lâu quá mặc_dù khách rất vắng đợi gần 15p mới mang được ly cà_phê ra ông này chỉ khéo quảng_bá chứ với mình chắc không quay lại_nữa
3,b45a1ff1,"Ko gian bé_tí , quán chật_chội , đông người nên phục_vụ chưa tốt . Đồ uống bình_thường ( đặc_biệt khi so với giá_cả ) và ko có gì đặc_sắc cả .",0,ko gian bé_tí quán chật_chội đông người nên phục_vụ chưa tốt đồ uống bình_thường đặc_biệt khi so với giá_cả và ko có gì đặc_sắc cả
4,f92694b0,"Khi mình order , đặt bánh thì nhận được sự tiếp_đãi , tư_vấn và nói_chuyện rất gần_gũi , rất nhiệt_tình .\nBánh thì rất tuyệt , nhân béo , rau_câu không dùng phẩm . Sau_này có dịp sinh_nhật , mình sẽ order mẫu lớn .\nChúc các bạn làm_ăn phát_đạt !",1,khi mình order đặt bánh thì nhận được sự tiếp_đãi tư_vấn và nói_chuyện rất gần_gũi rất nhiệt_tình bánh thì rất tuyệt nhân béo rau_câu không dùng phẩm sau_này có dịp sinh_nhật mình sẽ order mẫu lớn chúc các bạn làm_ăn phát_đạt
5,8fea8db2,"An ở đây lần thứ n mà giờ mới review . Quán lúc_nào cũng đông . Nhân_viên chạy ra chạy vô tội_nghiệp luôn . Mọi thứ viên chiên ở đây có đủ . Mỗi lần thèm an vặt là mình lại ra đây . Giá khá rẻ . 1 dĩa an phủ_phê luôn 4 đứa chia ra mỗi đứa 50k . Lúc trước an thì quán chỉ bán ở nhà chính thôi , riết rồi lấn ra đuờng rồi qua bên quán cơm tấm 3 lùn bán luôn . Phát_triển ghê_gớm",0,an ở đây lần thứ n mà giờ mới review quán lúc_nào cũng đông nhân_viên chạy ra chạy vô tội_nghiệp luôn mọi thứ viên chiên ở đây có đủ mỗi lần thèm an vặt là mình lại ra đây giá khá rẻ 1 dĩa an phủ_phê luôn 4 đứa chia ra mỗi đứa 50k lúc trước an thì quán chỉ bán ở nhà chính thôi riết rồi lấn ra đuờng rồi qua bên quán cơm tấm 3 lùn bán luôn phát_triển ghê_gớm
6,2db8ae0a,"Quán rất đẹp . Nhạc rất hay . Rất art ! ! Bước vô là thích liền luôn : 3 Đồ uống thì buổi_sáng hôm đó mình đi thì_có cà_phê à , nên uống bạc xỉu đỡ : ' ( phục_vụ dễ_thương . Nói_chung quán yên_tĩnh và đẹp lắm , mình thích nhưng nơi yên_tĩnh như_vậy . Sẽ nhất_định quay lại Người_Tình . À , trên bàn dài có cuốn note bự thiệt bự , dành cho mọi người thích ghi những kỉ_niệm đến quán hay của riêng mình vào ... Có nhiều người vẽ đẹp lắm , hi . Khen mãi quán thôi , chả có gì chê luôn . = ) )",1,quán rất đẹp nhạc rất hay rất art bước vô là thích liền luôn 3 đồ uống thì buổi_sáng hôm đó mình đi thì_có cà_phê à nên uống bạc xỉu đỡ phục_vụ dễ_thương nói_chung quán yên_tĩnh và đẹp lắm mình thích nhưng nơi yên_tĩnh như_vậy sẽ nhất_định quay lại người_tình à trên bàn dài có cuốn note bự thiệt bự dành cho mọi người thích ghi những kỉ_niệm đến quán hay của riêng mình vào có nhiều người vẽ đẹp lắm hi khen mãi quán thôi chả có gì chê luôn
7,5a840374,"Gemini thì nhạc bao hay òy : 3 Dj đánh là lên nóc luôn ... Lên là lên là lên đụng nóc luônnn ... Cập_nhậ

In [19]:
data.to_csv('train_preprocess.csv', columns=['id', 'preprocess_text', 'class'])
!cp "train_preprocess.csv" $output_path

# Test Section

In [20]:
train_preprocess_path = "/content/drive/MyDrive/ML/BERT/train_preprocess.csv"

prep = pd.read_csv(train_preprocess_path)
data['teacher_preprocess_text'] = prep['preprocess_text']

In [21]:
import numpy as np

data['compare'] = np.where((data['preprocess_text'] == data['teacher_preprocess_text']), 1, 0)
data['compare'].value_counts()

1    15307
0    11693
Name: compare, dtype: int64

In [22]:
# Test 1 sample

id = 14

ori_text = data['text'][id]
trans_text = preprocess_text(ori_text)
true_text = data['teacher_preprocess_text'][id]

print(f"TRA >>{trans_text}<<")
print(f"TRU >>{true_text}<<")
print()
print(f"ORI >>{ori_text}<<")

TRA >>mình đãđến đây với ýđịnh đến cho biết nhưng không ngờber vàđồăn ởđây lại tuyệt với đến vậy không_gian cũng rất làđẹp với sựquan_tâm chu đáo của nhân_viên mình đãtìm được loại ber mình thích trong hơn 10 loại ber ởđây vàđây cũng chính làlần đầu tiên mình được uống những chai ber ngon như thếởviệt nam cuối cùng mình cũng tìm được một địa điểm đểthư giản tại sài thành xô bồnày<<
TRU >>mình đã đến đây với ý_định đến cho biết nhưng không ngờ ber và đồ ăn ở đây lại tuyệt với đến vậy không_gian cũng rất là đẹp với sự quan_tâm chu_đáo của nhân_viên mình đã tìm được loại ber mình thích trong hơn 10 loại ber ở đây và đây cũng chính là lần đầu_tiên mình được uống những chai ber ngon như_thế ở việt nam cuối_cùng mình cũng tìm được một địa_điểm để thư_giản tại sài thành xô_bồ này<<

ORI >>Mi ̀ nh đa ̃ đê ́ n đây vơ ́ i y ́ đi ̣ nh đê ́ n cho biê ́ t nhưng không ngơ ̀ beer va ̀ đô ̀ ăn ơ ̉ đây la ̣ i tuyê ̣ t vơ ́ i đê ́ n vâ ̣ y .

In [23]:
# Show random

data.sample(20)

,id,text,class,preprocess_text,teacher_preprocess_text,compare
2561,b3616644,"Đồ_ăn o đây quá ngon , rất vừa_miệng và phuc vu chu_đáo . view nhin ra rat dep ngay trung_tâm , co nhiều nước lạ và ngon . khi ăn món thái sẽ ghé nơi này thuong xuyen",1,đồ_ăn o đây quá ngon rất vừa_miệng và phuc vu chu_đáo view nhin ra rat dep ngay trung_tâm co nhiều nước lạ và ngon khi ăn món thái sẽ ghé nơi này thuong xuyen,đồ_ăn o đây quá ngon rất vừa_miệng và phuc vu chu_đáo view nhin ra rat dep ngay trung_tâm co nhiều nước lạ và ngon khi ăn món thái sẽ ghé nơi này thuong xuyen,1
23228,80c52812,"Hôm_qua có cùng 3 nhỏ bạn vào uống trà sữa . Mình gọi 4 ly . Thêm trân_châu , thạch này_nọ xong mình quay ra thì con bấm biêu kêu là đụ_mẹ kêu gì nhìu v . Lúc đó mình rất bực nhưng do đông nên mình khôg nói_gì . Còn nước thì pha dở . Ngọt gắt . Giữ xe thì_phải dắt xe cho khách này chạy lên xong kêu xuống chạy qua kia để đi rồi ngồi bấm điện_thoại @ @ . Nói_chung trong tất_cả bobapop chỉ có pxl là tệ nhất từ phục_vụ cho tới nước pha",0,hôm_qua có cùng 3 nhỏ bạn vào uống trà sữa mình gọi 4 ly thêm trân_châu thạch này_nọ xong mình quay ra thì con bấm biêu kêu là đụ_mẹ kêu gì nhìu v lúc đó mình rất bực nhưng do đông nên mình khôg nói_gì còn nước thì pha dở ngọt gắt giữ xe thì_phải dắt xe cho khách này chạy lên xong kêu xuống chạy qua kia để đi rồi ngồi bấm điện_thoại nói_chung trong tất_cả bobapop chỉ có pxl là tệ nhất từ phục_vụ cho tới nước pha,hôm_qua có cùng 3 nhỏ bạn vào uống trà sữa mình gọi 4ly thêm trân_châu thạch này_nọ xong mình quay ra thì con bấm biêu kêu là đụ_mẹ kêu gì nhìu v lúc đó mình rất bực nhưng do đông nên mình khôg nói_gì còn nước thì pha dở ngọt gắt giữ xe thì_phải dắt xe cho khách này chạy lên xong kêu xuống chạy qua kia để đi rồi ngồi bấm điện_thoại nói_chung trong tất_cả bobapop chỉ có pxl là tệ nhất từ phục_vụ cho tới nước pha,0
1798,09235bfe,"Bữa đọc review thấy nhìu người khen chỗ này bánh theo phong cách_nhật ngon nên mình chạy lại thử đúng là_hơi khó tìm thật 😄 nhưng tâm_hồn ăn_uống đã trổi dậy_thì quyết tìm cho ra chỗ này . Mình oder 9 viên nama matcha , sakura , socola den 78k , một miếng sakura cheese cake 55k , một ly trà đen với lá và hoa anh_đào 50k và xin review như sau :\nnama chocolate vị trà xanh ăn được , vị sakura hơi ngọt nhưng cũng ngon , vị socola đen ăn vào phần bột ở trên rất đắng nếu muốn ăn phải vẩy bỏ phần bột ở trên không đắng lắm , socola này tan nhanh nên phải ăn hết không là nó tan ra liền 😐\nsakura cheese cake ăn ngon mềm mịn lại không quá ngọt vị anh_đào rất thích 😘\ntrà đen với lá và hoa anh_đào nghe tên thì hay vậy thôi chứ lúc đem ra không thấy anh_đào đâu mình bỏ 5 viên đường nhỏ ( lúc đầu 2 viên sau nếm còn lạt thêm 3 viên cũng chỉ ngọt 1 chút_xíu thôi ) vào rồi mà vẫn còn lạt không thơm vị trà uống lạt mà cũng không có vị hoa anh_đào luôn nói_đúng_ra giống trà lipton thường không pha gì vậy ah . Lần sau ko oder nữa\nKhông_gian đẹp cafe biệt_thự nên giá cũng cao theo , giá khá cao\nPhục_vụ không được pro , mình oder 3 thứ mà ngồi gần 15 phút không thấy ra cho mình nước hay bánh gì cả phải nhắc lại 😑 tiệm này không có khăn giấy luôn hơi bất_tiện vì mình đi bên MOF cafe hay có khăn giấy cho khách dùng , cũng may mình có đem khăn giấy theo nhìn quanh thấy bàn nào cũng vậy 😕 chứ không riêng bàn mình không có .",0,bữa đọc review thấy nhìu người khen chỗ này bánh theo phong cách_nhật ngon nên mình chạy lại thử đúng là_hơi khó tìm thật nhưng tâm_hồn ăn_uống đã trổi dậy_thì quyết tìm cho ra chỗ này mình oder 9 viên nama matcha sakura socola den 78k một miếng sakura chese cake 5k một ly trà đen với lá và hoa anh_đào 50k và xin review như sau nama chocolate vị trà xanh ăn được vị sakura hơi ngọt nhưng cũng ngon vị socola đen ăn vào phần bột ở trên rất đắng nếu muốn ăn phải vẩy bỏ phần bột ở trên không đắng lắm socola này tan nhanh nên phải ăn hết không là nó tan ra liền sakura chese cake ăn ngon mềm mịn lại không quá ngọt vị anh_đào rất thích trà đen với lá và hoa anh_đào nghe tên thì hay vậy thôi chứ lúc đ